In [62]:
import ccxt
import websocket
import json
import pandas as pd

In [63]:
endpoint = 'wss://stream.binance.com:9443/ws'

In [64]:
our_msg = json.dumps({'method':'SUBSCRIBE','params': ['btcusdt@ticker'], 'id':1})

In [65]:
df = pd.DataFrame()
in_position = False
buyorders,sellorders = [], []

def on_open(ws):
    ws.send(our_msg)

def on_message(ws, message):
    global df, in_position, buyorders, sellorders
    out = json.loads(message)
    out = pd.DataFrame({'price':float(out['c'])}, index=[pd.to_datetime(out['E'],unit='ms')])
    df = pd.concat([df,out], axis=0)
    print(df)
    df = df.tail(5)
    last_price = df.tail(1).prices.values[0]
    sma_5 = df.price.rolling[5].mean().tail(1).values[0]
    if not in_position and last_price > sma_5:
        print('bought for '+ str(last_price))
        buyorders.append(last_price)
        in_position = True
    if in_position and sma_5 > last_price:
        print('sold for '+ str(last_price))
        print('profit: ' + str(last_price - buyorders[-1]))
        sellorders.append(last_price)
        in_position = False


In [66]:
ws = websocket.WebSocketApp(endpoint, on_message=on_message,
                            on_open=on_open)

ws.run_forever()

                            price
2023-02-28 03:41:53.598  23463.02
                            price
2023-02-28 03:41:53.598  23463.02
2023-02-28 03:41:54.600  23462.59
                            price
2023-02-28 03:41:53.598  23463.02
2023-02-28 03:41:54.600  23462.59
2023-02-28 03:41:55.596  23463.02
                            price
2023-02-28 03:41:53.598  23463.02
2023-02-28 03:41:54.600  23462.59
2023-02-28 03:41:55.596  23463.02
2023-02-28 03:41:56.601  23463.32
                            price
2023-02-28 03:41:53.598  23463.02
2023-02-28 03:41:54.600  23462.59
2023-02-28 03:41:55.596  23463.02
2023-02-28 03:41:56.601  23463.32
2023-02-28 03:41:57.600  23463.31
                            price
2023-02-28 03:41:53.598  23463.02
2023-02-28 03:41:54.600  23462.59
2023-02-28 03:41:55.596  23463.02
2023-02-28 03:41:56.601  23463.32
2023-02-28 03:41:57.600  23463.31
2023-02-28 03:41:58.598  23464.41
                            price
2023-02-28 03:41:54.600  23462.59
2023-02-28 03:

False

In [67]:
df.tail(1).price.values[0]

23459.88

In [68]:
df.price.rolling(5).mean()

2023-02-28 03:49:12.669          NaN
2023-02-28 03:49:13.673          NaN
2023-02-28 03:49:14.673          NaN
2023-02-28 03:49:15.672          NaN
2023-02-28 03:49:16.427    23457.486
Name: price, dtype: float64